In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("p2-texts/hansard40000.csv")
print(df)

                                                  speech         party  \
0      Unemployment is soaring, uptake in benefits ha...        Labour   
1      I thank the hon. Gentleman for raising issues ...  Conservative   
2      As my hon. Friend the Member for Portsmouth So...        Labour   
3      I thank the hon. Gentleman for raising the nee...  Conservative   
4      There is no doubt that the unemployment situat...        Labour   
...                                                  ...           ...   
39995  I totally agree with everything that the right...       Speaker   
39996  Message to attend the Lords Commissioners deli...           NaN   
39997  I have to acquaint the House that the House ha...       Speaker   
39998  I have further to acquaint the House that the ...       Speaker   
39999  The Commission was also for proroguing this pr...       Speaker   

                    constituency        date speech_class  \
0               Portsmouth South  2020-09-14      

In [3]:
df['party'] = df['party'].replace('Labour (Co-op)', 'Labour')
print(df['party'])

0              Labour
1        Conservative
2              Labour
3        Conservative
4              Labour
             ...     
39995         Speaker
39996             NaN
39997         Speaker
39998         Speaker
39999         Speaker
Name: party, Length: 40000, dtype: object
